In [ ]:
from typing import *
from itertools import product
from time import time

import numpy as np
import fastplotlib as fpl
import pygfx

In [ ]:
def make_circle(
    center: Tuple[float, float], radius: float, n_points: int = 50
) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)
    zs = np.zeros(xs.size)

    xs += center[0]
    ys += center[1]

    return np.ascontiguousarray(np.column_stack([xs, ys, zs]).astype(np.float32))

In [ ]:
spatial_dims = (1000, 1000)

circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

In [ ]:
len(circles)

In [ ]:
from wgpu.gui.auto import WgpuCanvas, run
import pygfx as gfx

In [ ]:
canvas = WgpuCanvas()
renderer = gfx.WgpuRenderer(canvas)

In [ ]:
scene = gfx.Scene()

In [ ]:
lines = list()

for c in circles:
    line = gfx.Line(
        gfx.Geometry(positions=c),
        gfx.LineMaterial(thickness=1.2, color=(1, 1, 1, 1)),
    )
    lines.append(line)
    scene.add(line)

In [ ]:
gfx.show(scene, camera=gfx.OrthographicCamera(1000, 1000))

In [ ]:
camera = gfx.OrthographicCamera(1000, 1000)
controller = gfx.PanZoomController(camera, register_events=renderer)

In [ ]:
t1 = time()

for l in lines[100:1000]:
    l.visible = False

# canvas.request_draw()

time() - t1

In [ ]:
temporal = list()

xs = np.arange(0, 10_000)
for i in range(len(circles)):
    if i % 2 == 0:
        ys = np.sin(xs) * 10
    else:
        ys = np.cos(xs) * 10

    temporal.append(ys)

In [ ]:
temporal = np.vstack(temporal)

In [ ]:
plot = fpl.GridPlot((1, 2))

# contours = plot[0, 0].add_line(np.vstack(circles), thickness=3)
contours = plot[0, 0].add_line_collection(circles, thickness=1.5)
heatmap = plot[0, 1].add_heatmap(temporal)
selector = heatmap.add_linear_region_selector(axis="y")

plot.show()

In [ ]:
def update_visible(ev):
    ixs_visible = ev.pick_info["selected_indices"]
    ixs_hide = np.setdiff1d(np.arange(len(circles)), ixs_visible)

    # very fast, 20 ms to change 1,000
    for i, g in enumerate(contours.graphics):
        if not g.visible and i in ixs_visible:
            g.visible = True
        elif g.visible and i in ixs_hide:
            g.visible = False

In [ ]:
selector.bounds.add_event_handler(update_visible)

In [ ]:
selector.bounds.remove_event_handler(update_visible)

In [ ]:
selector.bounds._event_handlers.clear()

In [ ]:
class Event:
    pick_info = {"selected_indices": np.arange(2000, 3000)}

In [ ]:
t1 = time()

update_visible(Event())

time() - t1

In [ ]:
t1 = time()

for c in contours.graphics[100:1000]:
    c.visible = True

# for i in range(100, 1000):
#     contours.graphics[i].world_object.visible = True

time() - t1

In [ ]:
zero_alpha_ixs = list()

start_offset = 0
for c in circles:
    start_offset += c.shape[0]
    zero_alpha_ixs += [start_offset - 1, start_offset]

zero_alpha_ixs = zero_alpha_ixs[:-1]

In [ ]:
contours.colors[np.array(zero_alpha_ixs)] = [1, 1, 1, 0]

In [ ]:
def set_visible_alpha(ev):
    ixs_visible = ev.pick_info["selected_indices"]
    ixs_hide = np.setdiff1d(np.arange(len(circles)), ixs_visible)

    for i in ixs_visible:
        contours.world_object.geometry.colors.data[(i * 75) + 1 : (i * 75) + 74, -1] = 1

    for i in ixs_hide:
        contours.world_object.geometry.colors.data[(i * 75) + 1 : (i * 75) + 74, -1] = 0

    contours.world_object.geometry.colors.update_range()

In [ ]:
selector.bounds.add_event_handler(set_visible_alpha)